In [1]:
import pandas as pd
from datetime import datetime
import sys
import re
from tika import parser #parse pdf text
import numpy as np
from empath import Empath # similar to LIWC
import math
pd.options.mode.chained_assignment = None 

This script generates posting frequency feature, empath and dictionary feature
for posting frequency, see example out[303]
postingFrequency: number of posts in this file
postingInterval: intervals between previous and later posts (days) 
generalMoreFreq: is the posting behavior becomes more frequent
generalWordCount: mean word count of the posts in each user
Note: these features are built on user level

We can also select posts than contain certain diction see out[306]
MH is a file with posts contain mentall illness, symptoms keywords 
then we compute the posting behaviors in regard to these posts
the features are:
healthPostingFrequency	healthPostingInterval	healthMoreFreq

Empath In[501]
this is the empath features on post level 

In [2]:
path = '/Users/lucia/phd_work/suicideDetection/'
#path = '/home/lucia/phd_work/shareTask/CLpsych/'

In [10]:
pos1 = pd.read_csv(  path +'/data_sample_clpsych19/user10146.posts.csv')
pos2 = pd.read_csv(  path +'/data_sample_clpsych19/user10276.posts.csv')
neg1 = pd.read_csv( path +'/data_sample_clpsych19/user-29175.posts.csv')
neg2 = pd.read_csv( path +'/data_sample_clpsych19/user-44823.posts.csv')
file = pos1

In [11]:
# file1 = pd.concat([pos1, pos2], ignore_index= True)
# file2 = pd.concat([neg1, neg2], ignore_index= True)
# file = pd.concat([file1, file2], ignore_index= True) 
#convert timestamp to time, sorted the table according to time 
file['timestamp'] = file['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
file[1:5]



,post_id,user_id,timestamp,subreddit,post_title,post_body
1,275dra,10146,2014-06-02 22:05:47,OpTicGaming,[Throwback: BO2 MLG Anaheim] WHO'S THE BEST IN...,NaN
2,2760t9,10146,2014-06-03 02:12:28,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"What's going on everybody? My name is Waf3l, b..."
3,277upp,10146,2014-06-03 17:33:23,CoDCompetitive,Who is your favorite Pro Team?,"I know we have the flair to back it up, but I ..."
4,27cmpe,10146,2014-06-05 02:53:13,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","What's going on everybody? My name is Waf3l, b..."


we build features on user level, for each user, we have Number of annotation unit, number of posts in sucideWatch, average length of post in suicideWatch, frequency of posting in all subreddit, frequency of posting in suicideWatch, posts mentioned mental illness, sentiment in posts (LIWC), number of posts without content, posts that mention suicide methods(tools)

Now we create a dictionary that stores the posting pattern 

In [12]:
def PostingFreq(file):
    timeFreq = {} #dictionary that shows the average time a user post sth 
    preUser = None #previous user 
    preTime = None #posting time of the previous day
    dayVec = [] #a vector that shows the time difference between a previous day and the day after
    
    for userid, time in zip(file['user_id'], file['timestamp']):
        if preTime is None:
            freq = time 
        elif userid == preUser:
            #freq = previous day - day after
            freq = datetime.strptime(time, "%Y-%m-%d %H:%M:%S") - datetime.strptime(preTime, "%Y-%m-%d %H:%M:%S")
            dayVec.append(freq.days) 
            timeFreq[userid] = dayVec
            
        elif type(freq) is not str: # freq is not a datetime object if there is one case only
            dayVec = [freq.days] 

        preTime = time 
        preUser = userid
        
    return timeFreq

postingFrequency = PostingFreq(file)


In [6]:
for item in postingFrequency:
    print(postingFrequency[item])

[3, 0, 0, 1, 0, 3, 8, 1, 0, 0, 1, 0, 0, 4, 1, 32, 0, 2, 25, 17, 6, 7, 3, 5, 28, 3, 0, 6, 70, 56, 2, 0, 4, 89, 3, 12, 9]
[9, 0, 0, 4, 1, 7, 5, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 5, 6, 5, 1, 4, 0, 0, 0, 1, 5, 2, 4, 0, 3, 36, 2, 1, 1, 0, 5, 9, 2, 1, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 10, 4, 1, 19, 0, 19, 23, 34, 14, 8, 12, 23, 70, 1, 0, 1, 0, 0, 7, 8, 2, 2, 6, 3, 9, 11, 6, 0, 0, 0, 0, 0, 0, 0, 8, 12, 1, 9, 4, 1, 0, 0, 13, 3, 0, 27, 2, 2, 4, 21, 1, 11, 20, 0, 13, 43, 3, 4, 9, 1, 2, 2, 4, 48, 25, 2, 9, 7, 0, 2, 11, 10, 2, 70, 114, 1, 1, 12, 11, 148]
[148, 2, 1, 0, 26, 612, 30, 67, 122, 0, 19, 0, 8, 2, 5, 2, 0, 22]
[22, 0, 1, 0, 24, 14, 22, 10, 6, 6, 63, 107, 112, 17, 36, 4, 1, 0, 20, 22, 0, 12, 0, 37, 0, 20, 3, 100, 15, 18, 24, 32, 335, 33, 8, 31, 137, 162, 0, 11, 1, 7, 9, 3, 0, 0, 0, 0, 4, 2, 0, 8, 5, 2, 5, 4, 0, 6, 8, 3, 2, 2, 0, 7, 10, 2, 0, 3, 6, 0, 5, 1, 0, 16]


In [13]:
#here we compute the mean of posting interval 
def computeMean(dictionary):
    mean = []
    for i in dictionary:
        mean.append((sum(dictionary[i])/len(dictionary[i])))
    return mean
        
mean = computeMean(postingFrequency)
mean

[10.837837837837839]

In [14]:
#we can also get the frequency 
def computeFreq(dictionary):
    freq = []
    for i in dictionary:
        freq.append(len(dictionary[i])+1)
    return freq

freq = computeFreq(postingFrequency)
freq

[38]

The interval mean does not tell whether the user have increased the posting frequency, so here we will see if the posting behaviour is getting more frequent. define a sliding window to compare frequencies, we can define the size of the window, if the last window (closest to data extaction time) is 1 sd below the mean, we assign '1' to indicate the author post more frequent recently

In [15]:
timeInterval = [8, 7, 6, 5, 4, 3, 2, 1, 0]
def getSlidingWindow(array, window):
    ret = []
    i = 0
    j = window
    while j < len(array):
        ret.append((sum(array[i:j]), i,j))
        i = i+1
        j = j+1
    ret.sort()
    ret.reverse()
    return ret

#if the last time window is 1sd below the mean
def isMoreFrequent(array, window):
    meanlist = []
    for item in getSlidingWindow(array, window):
        meanlist.append(item[0])
    meanL = np.mean(meanlist)
    std = np.std(meanlist)
    result = []
    indexL = []
    for index, item in enumerate(getSlidingWindow(array, window)):
        
        if item[0]  <= meanL - std:
            indexL.append(index)       
    length = len(getSlidingWindow(array, window))-1
    if length != 0:
        if length in indexL:
            return 1
        else:
            return 0
        
def getMoreFrequent(postingFrequency, window):
    results = []
    for item in postingFrequency:
        result = isMoreFrequent(postingFrequency[item],window)
        results.append(result)
    return results

def stringCount(text):
    count = 0
    for item in text.split():
        count = count + 1
    return count 

#this function return a frequency feature matrix 
def getFrequencyFeature(file, intervalName, freqName, isMoreFreq, PostWordCount, SlideWindow): #input df that you need to count the posting frequency and interval 
    #dictionary that shows the average time a user post sth 
    postingFrequency = PostingFreq(file)
    #mean of posting interval
    mean = computeMean(postingFrequency)
    # is the posting behaviour becoming more frequent
    MoreFreq = getMoreFrequent(postingFrequency, SlideWindow)
    #comupte number of postings
    freq = computeFreq(postingFrequency)
    #mean wordcount in post
    PostCount = file.groupby(['user_id']).size().reset_index(name = 'counts')
    file = pd.merge(PostCount, file, on='user_id')
    MoreThanOnePost = file[file.counts > 1]
    MoreThanOnePost['wordCount'] = MoreThanOnePost['post_body'].apply(lambda x: stringCount(str(x)))
    wordCount = MoreThanOnePost.groupby(['user_id'])['wordCount'].mean().reset_index(name = 'counts')
    
    #append all the features
    ProtoDf = np.vstack((list(postingFrequency.keys()),freq))
    ProtoDf = np.vstack((ProtoDf, mean))
    ProtoDf = np.vstack((ProtoDf, MoreFreq))
    ProtoDf = np.vstack((ProtoDf, wordCount.counts))
    df = pd.DataFrame(ProtoDf)
    featureTable = df.T 
    featureTable.columns = ['user_id',freqName,intervalName, isMoreFreq, PostWordCount]
    
    #this part only triggers when there are users posted 1 post
    onePost = PostCount[PostCount.counts == 1]
    one = file[file.counts == 1]
    #now create a table with this one post id, assign NaN to interval and 1 to posting Frequency
    if one.empty is False: 
        one['wordCount'] = one['post_body'].apply(lambda x: stringCount(str(x)))
        onePost[intervalName] = 0
        onePost[isMoreFreq] = 0
        onePost[PostWordCount] = one.wordCount
        onePost.columns = ['user_id', freqName, intervalName, isMoreFreq, PostWordCount]
        featureTable = featureTable.append(onePost,ignore_index=True)
    
    return featureTable

FreqTable = getFrequencyFeature(file, 'postingInterval','postingFrequency', 'generalMoreFreq', 'generalWordCount', 6)
FreqTable


,user_id,postingFrequency,postingInterval,generalMoreFreq,generalWordCount
0,10146.0,38.0,10.837838,1.0,83.473684


In [16]:
file['wordCount'] = file['post_body'].apply(lambda x: stringCount(str(x)))
wordCount = file.groupby(['user_id'])['wordCount'].mean().reset_index(name = 'counts')
wordCount.count

<bound method DataFrame.count of    user_id     counts
0    10146  83.473684>

check posting behavior on different subreddits

In [17]:
suicideWatch = file[file['subreddit'].str.contains("SuicideWatch")]
suicideWatch

,post_id,user_id,timestamp,subreddit,post_title,post_body,wordCount
14,2985q5,10146,2014-06-27 08:15:28,SuicideWatch,I'm done.,I'm just done with life. I'm nothing but a scr...,80


In [18]:
getFrequencyFeature(suicideWatch, 'SuicidePostingInterval','SuicidePostingFrequency', 'SuicideMoreFreq', 'SuicideWordCount', 3)

DataError: No numeric types to aggregate

Now we create a dictionary with all mental illness names and symptoms, we downloaded a pdf file from file:///home/lucia/phd_work/shareTask/CLpsych/glossary-mental-health-i.pdf 
we extract the terms (starts with capital letter), then we mannually selected the terms and save it as dictionary

In [38]:
#read pdf file 
raw = parser.from_file(path+ '/dictionaries/glossary-mental-health.pdf')
t = re.findall('([A-Z][a-z]+)', raw['content'])
Psylist = set(t)
#save list as txt
# with open("/home/lucia/phd_work/shareTask/CLpsych/psyList.txt", "w") as output:
#     output.write(str(Psylist))


In [41]:
#read adjusted mental illness glossary list
def readDictionaries(filePath):
    with open(filePath) as f:
        myList = [x.strip().replace("'","").lower() for x in f.read().split(",")]
        #print(Psylist)
    return myList

PsyList = readDictionaries(path +'/dictionaries/psyList.txt')

In [42]:
PsyList[0]

'opioid'

In [43]:
#clean text, lower cases
file['post_title'] = file['post_title'].apply(lambda x: x.lower())
file['subreddit'] = file['subreddit'].apply(lambda x: x.lower())
file['post_body'] = file['post_body'].apply(lambda x: x.lower() if type(x) is str else 'NULL')
file[0:3]

,post_id,user_id,timestamp,subreddit,post_title,post_body,wordCount,Psy_title
0,26u0eb,10146,2014-05-30 00:13:59,dating,am i doing something wrong?,"as the title says, i have no clue what i'm doi...",141,False
1,275dra,10146,2014-06-02 22:05:47,opticgaming,[throwback: bo2 mlg anaheim] who's the best in...,NULL,1,False
2,2760t9,10146,2014-06-03 02:12:28,foreveralonedating,20 [m4f] wa - looking for the player 2 to my p...,"what's going on everybody? my name is waf3l, b...",611,False


In [44]:
#if words find in text, print 'T', otherwise 'F'
def findText(text, wordList):
    for item in wordList:
        if item in text:
            return True
    return False

def subsetDictPosts(file, dictionary, newTitle, newBody, newSubreddit):
    file[newTitle] = file.apply(lambda row: True if findText(row["post_title"], dictionary) else False, axis=1)
    file[newBody] = file.apply(lambda row: True if findText(row["post_body"], dictionary) else False, axis=1)
    file[newSubreddit] = file.apply(lambda row: True if findText(row["subreddit"], dictionary) else False, axis=1)

subsetDictPosts(file, PsyList, 'Psy_title', 'Psy_body', 'Psy_subre')
file[0:3]

,post_id,user_id,timestamp,subreddit,post_title,post_body,wordCount,Psy_title,Psy_body,Psy_subre
0,26u0eb,10146,2014-05-30 00:13:59,dating,am i doing something wrong?,"as the title says, i have no clue what i'm doi...",141,False,False,False
1,275dra,10146,2014-06-02 22:05:47,opticgaming,[throwback: bo2 mlg anaheim] who's the best in...,NULL,1,False,False,False
2,2760t9,10146,2014-06-03 02:12:28,foreveralonedating,20 [m4f] wa - looking for the player 2 to my p...,"what's going on everybody? my name is waf3l, b...",611,False,False,True


In [45]:
#these are the posts contain words in the dictionary
MH = file[(file['Psy_body'] == True) | (file['Psy_title'] == True) | (file['Psy_subre'] == True)]
MH.shape

(31, 10)

In [46]:
#now we see whether users post mental illness related things more frequent, we can merge this with feature matrix
#later on 
MHF = getFrequencyFeature(MH, 'healthPostingInterval','healthPostingFrequency', 'healthMoreFreq', 'healthWordCount',4)
MHF

/Users/lucia/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/lucia/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/lucia/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/lucia/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/lucia/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,user_id,healthPostingFrequency,healthPostingInterval,healthMoreFreq,healthWordCount
0,10146.0,4.0,36.666667,0.0,173.250000
1,10276.0,28.0,22.000000,1.0,77.777778


In [55]:
#now we can merge the features 
FreqFea = pd.merge(FreqTable, MHF, on ='user_id', how = 'left')

Check suicide methods, we created a dictionary suicideMethods.txt to check methods and tools mention in the posts

In [56]:
methods = readDictionaries(path + '/dictionaries/suicideMethods.txt')
subsetDictPosts(file, methods, 'med_title', 'med_body', 'med_subre')
#we only check if suicidewatch posts contain these keywords
suicideWatch = file[file['subreddit'] == 'suicidewatch']
med = suicideWatch[(suicideWatch['med_body'] == True) | (suicideWatch['med_title'] == True)]
med

,post_id,user_id,timestamp,subreddit,post_title,post_body,wordCount,Psy_title,Psy_body,Psy_subre,med_title,med_body,med_subre
14,2985q5,10146,2014-06-27 08:15:28,suicidewatch,i'm done.,i'm just done with life. i'm nothing but a scr...,80,False,False,True,False,True,False


In [57]:
def checkDictFea(file):
    file.drop_duplicates(subset=['user_id'], keep=False)
    newFea = file[['user_id']]
    newFea['mentionMethods'] = 1
    return newFea

methods = checkDictFea(med)    

In [58]:
FreqFea = pd.merge(FreqFea, methods, on ='user_id', how = 'left')
FreqFea.fillna(0, inplace=True)
FreqFea.to_csv(path+'FreqFea.csv')
FreqFea

,user_id,postingFrequency,postingInterval,generalMoreFreq,generalWordCount,healthPostingFrequency,healthPostingInterval,healthMoreFreq,healthWordCount,mentionMethods
0,10146.0,38.0,10.837838,1.0,26.864865,4.0,36.666667,0.0,173.250000,1.0
1,10276.0,154.0,7.352941,0.0,13.055556,28.0,22.000000,1.0,77.777778,0.0
2,-29175.0,19.0,59.222222,1.0,83.473684,0.0,0.000000,0.0,0.000000,0.0
3,-44823.0,75.0,21.432432,0.0,26.183007,0.0,0.000000,0.0,0.000000,0.0


Here we will generate sentiment and topic features for each post

In [79]:
lexicon = Empath()
def getEmpath(file):
    empathy = {}
    for item, postid in zip(file['post_body'],file['post_id']):
        empathy[postid] = lexicon.analyze(item, normalize=True) 
    empathyFea = pd.DataFrame.from_dict(empathy)
    empathyFea = empathyFea.T
    empathyFea['post_id'] = empathyFea.index
    return empathyFea

def getDictionsEmpath(file, PsyList, EmpathVars):
    subsetDictPosts(file, PsyList, 'Psy_title', 'Psy_body', 'Psy_subre')
    subset = file[(file['Psy_body'] == True) | (file['Psy_title'] == True) | (file['Psy_subre'] == True)]
    Features = getEmpath(subset)
    Fea = pd.merge(subset, Features, on = 'post_id', how ='left')
    empathFea = Fea[EmpathVars]
    #aggregate to user level 
    empath = empathFea.groupby('user_id').mean()
    return empath

#define dictionary and the empath topics you want to retain
EmpathVars = ['post_id','user_id', 'achievement', 'shame', 'affection', 'aggression', 'anger', 'cheerfulness', 'disappointment', 'disgust', 'dispute', 'emotional', 'fear', 'fun', 'hate', 'joy', 'love', 'negative_emotion', 'nervousness', 'optimism', 'pain', 'positive_emotion', 
                 'suffering', 'alcohol', 'appearance', 'attractive', 'banking', 'money', 'body', 'health', 'injury', 'medical_emergency', 'death', 'friends', 'help']
fea = getDictionsEmpath(file, PsyList, EmpathVars)
fea

,achievement,shame,affection,aggression,anger,cheerfulness,disappointment,disgust,dispute,emotional,...,attractive,banking,money,body,health,injury,medical_emergency,death,friends,help
user_id,,,,,,,,,,,,,,,,,,,,,
10146,0.0,0.006250,0.002455,0.000818,0.000000,0.0,0.000000,0.0,0.000409,0.000000,...,0.004762,0.000409,0.000409,0.000000,0.000818,0.00000,0.000818,0.003534,0.007217,0.001227
10276,0.0,0.002518,0.000123,0.000000,0.000123,0.0,0.000311,0.0,0.000313,0.001335,...,0.000428,0.000544,0.000544,0.002519,0.003040,0.00147,0.003562,0.002655,0.001384,0.002654


skip-gram/infersent preserve knowledge in sentence level, cluster the posts with sentence vector

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, f1_score, precision_score,\
recall_score, confusion_matrix, classification_report, accuracy_score 
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn import svm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sys import argv
import gc
from sklearn.decomposition import TruncatedSVD
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.combine import SMOTEENN

X_train, X_test, y_train, y_test = train_test_split(x_vec, y, test_size=0.30, random_state=30)
###grid search
#SMOTE: Synthetic Minority Over-sampling Technique
smote_enn = SMOTEENN(random_state=42)
cv = StratifiedKFold(n_splits=5, random_state = 0)
clf = make_pipeline(smote_enn, SGDClassifier(max_iter= 1000),StandardScaler())
parameters = [{'sgdclassifier__alpha': [0.01, 0.05, 0.001, 0.005], 'sgdclassifier__class_weight':['balanced'],
                'sgdclassifier__loss': ['hinge','log','modified_huber','squared_hinge', 'perceptron'], 
                 'sgdclassifier__penalty':['none','l1','l2']}]                   
grid_search_item = GridSearchCV(estimator = clf,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print('Best scores and best parameters')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_true, y_pred = y_test, grid_search.predict(X_test)
print(classification_report(y_true, y_pred))
  

print('Done!')


##Naive bayes
Create a Gaussian Classifier
model = GaussianNB()
If continuous features do not have normal distribution, we should use transformation or different methods to convert it in normal distribution.
If test data set has zero frequency issue, apply smoothing techniques “Laplace Correction” to predict the class of test data set.
Remove correlated features, as the highly correlated features are voted twice in the model and it can lead to over inflating importance.

Try
Bernoulli Naive Bayes 
Features represented by the presence of a term 